In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pycalphad import Database, calculate, equilibrium, variables as v
from pycalphad.plot.utils import phase_legend
from pycalphad import ternplot
import plotly.graph_objects as go
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from symbolic_boundary import *

import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

In [ ]:
# Load database and choose the phases that will be plotted
db = Database(r'../TDDatabaseFiles_temp/alfe.tdb')

phases = list(db.phases.keys())
constituents = list(db.elements)
legend_handles, color_dict = phase_legend(phases)

print(phases)
print(constituents)

### First we are going to compute all energy values in PyCalPhad and plot the phase diagram

In [ ]:
# This function is to format the result
def format_result(result):
    X = result.X.sel(component='FE').values[0, 0, 0, :]
    Y = result.T.values
    Z = result.GM.values

    sort_idx = np.argsort(X)
    X_sorted = X[sort_idx]
    Z_sorted = Z[:, :, :, sort_idx]

    sort_idx = np.argsort(Y)
    Y_sorted = Y[sort_idx]
    Z_sorted = Z_sorted[:, :, sort_idx, :]

    X, Y = np.meshgrid(X_sorted, Y_sorted)

    X = X.flatten()
    Y = Y.flatten()
    Z = Z_sorted.flatten()

    return X, Y, Z

In [ ]:
# This function is to format the result
def format_result(result):
    X = result.X.sel(component='FE').values[0, 0, 0, :]
    Y = result.T.values
    Z = result.GM.values

    sort_idx = np.argsort(X)
    X_sorted = X[sort_idx]
    Z_sorted = Z[:, :, :, sort_idx]

    sort_idx = np.argsort(Y)
    Y_sorted = Y[sort_idx]
    Z_sorted = Z_sorted[:, :, sort_idx, :]

    X, Y = np.meshgrid(X_sorted, Y_sorted)

    X = X.flatten()
    Y = Y.flatten()
    Z = Z_sorted.flatten()

    return X, Y, Z

phase_dict = dict()
for phase_name in phases:
    # Only computing 10 teperature points because the plotting struggles
    result = calculate(db, constituents, phase_name, P=101325, T=np.linspace(300, 2000, 10), output='GM')
    selected_color=color_dict[phase_name]
    X, Y, Z = format_result(result)

    phase_dict[phase_name] = (X, Y, Z)

In [ ]:
# # Make example phase dict and color dict
# x, y = sp.symbols('x y')

# # define your funcitons here in explicit form
# f1 = x**2 + y**2
# f2 = (x-1)**2 + (y+4)**2 + 1
# f3 = (x+3)**2 + (y+2)**2 + 1

# X = np.linspace(-5, 5, 300)
# Y = np.linspace(-5, 5, 300)
# X, Y = np.meshgrid(X, Y)

# Z1 = sp.lambdify((x, y), f1)(X, Y)
# Z2 = sp.lambdify((x, y), f2)(X, Y)
# Z3 = sp.lambdify((x, y), f3)(X, Y)

# x_flat = X.flatten()
# y_flat = Y.flatten()
# z1_flat = Z1.flatten()
# z2_flat = Z2.flatten()
# z3_flat = Z3.flatten()

# phase_dict = {
#     'alpha': (x_flat, y_flat, z1_flat),
#     'beta': (x_flat, y_flat, z2_flat),
#     'gamma' : (x_flat, y_flat, z3_flat)
# }

# color_dict = {'alpha': 'red',
#     'beta': 'blue',
#     'gamma' : 'green'
# }

In [ ]:
# # Plot the fake phases
# fig = plt.figure(figsize=(10, 8))
# ax = fig.add_subplot(111, projection='3d')

# for phase, (x_vals, y_vals, z_vals) in phase_dict.items():
#     ax.scatter(x_vals, y_vals, z_vals, c=color_dict[phase], label=phase, alpha=0.6, s=1)

# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.set_title('3D Scatter Plot of Phases')
# ax.legend()

# plt.show()

In [ ]:
# import numpy as np
# import plotly.graph_objects as go

# def plot_3d_hull_plotly(all_points, simplices):
#     """
#     all_points : (N, 3) ndarray of 3D coordinates
#     simplices  : list or array of simplices, each simplex is
#                  a tuple/list/array of vertex indices (e.g. [i0, i1, i2])
#     """
#     # Separate out coordinates
#     x = all_points[:, 0]
#     y = all_points[:, 1]
#     z = all_points[:, 2]

#     # Extract i, j, k from each simplex
#     i_vals = []
#     j_vals = []
#     k_vals = []
#     for simplex in simplices:
#         i_vals.append(simplex[0])
#         j_vals.append(simplex[1])
#         k_vals.append(simplex[2])

#     # Create a 3D mesh with Plotly
#     mesh = go.Mesh3d(
#         x=x, y=y, z=z,
#         i=i_vals, j=j_vals, k=k_vals,
#         opacity=0.5,
#         color='cyan',
#         # You can color each face separately by passing in intensity values:
#         # intensity = some_array_of_same_length_as_i_vals,
#         # colorscale='Viridis'
#     )

#     fig = go.Figure(data=[mesh])

#     # Optionally add the raw points as well
#     fig.add_trace(go.Scatter3d(
#         x=x, y=y, z=z,
#         mode='markers',
#         marker=dict(size=3, color='black', opacity=0.8),
#         name='Points'
#     ))

#     # Set up axes and layout
#     fig.update_layout(
#         title='3D Convex Hull',
#         scene=dict(
#             xaxis_title='X',
#             yaxis_title='Y',
#             zaxis_title='Z',
#             aspectmode='data'  # keeps the same scale for all axes
#         ),
#         width=800,
#         height=600
#     )

#     fig.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def plot_3d_hull(all_points, simplices):
    """
    all_points: (N, 3) array of 3D coordinates
    simplices: list (or array) of simplices, where each simplex is
               a list/tuple/array of indices into all_points
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Collect triangular faces
    faces = []
    for simplex in simplices:
        # simplex is something like [i0, i1, i2]
        tri_points = all_points[simplex]
        # Poly3DCollection expects a list of vertex arrays
        faces.append(tri_points)

    # Create a 3D polygon collection
    hull_poly = Poly3DCollection(faces, alpha=0.5, facecolors='cyan', edgecolors='k')
    ax.add_collection3d(hull_poly)

    # Optionally set axis limits to fit the data
    xs, ys, zs = all_points[:, 0], all_points[:, 1], all_points[:, 2]
    ax.set_xlim([xs.min(), xs.max()])
    ax.set_ylim([ys.min(), ys.max()])
    ax.set_zlim([zs.min(), zs.max()])

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('3D Convex Hull')
    ax.view_init(elev=0, azim=-90)

    plt.show()


In [ ]:
def plot_phase_diagram(phase_dict, color_dict):
    """
    Plot the 2D projection of a convex hull phase diagram.

    Parameters
    ----------
    phase_points : dict
        Dictionary mapping phase names to numpy arrays of points in (X, T, G) space.
        Each array can be provided as either shape (n_points, 3) or (3, n_points) or even have extra dimensions;
        they will be squeezed and transposed if needed.
        
    color_dict : dict
        Dictionary mapping phase names to color strings.

    Returns
    -------
    None
        Displays a matplotlib plot.
    """
    
    # Combine all phase points into a single data array and record their phase labels
    all_points = []
    phase_labels = []

    for phase, points in phase_dict.items():
        x_points = points[0].flatten()
        y_points = points[1].flatten()
        z_points = points[2].flatten()

        points = np.vstack((x_points, y_points, z_points)).T
        all_points.append(points)
        phase_labels.extend([phase] * x_points.shape[0])

    # Create a single NumPy array of all points and an array of phase labels.
    all_points = np.vstack(all_points)
    phase_labels = np.array(phase_labels)
    
    
    # Compute the lower convex hull using your provided function.
    # It should return an iterable of simplices (each simplex is an array of indices into all_points).
    simplices = lower_convex_hull(all_points)
    
    plot_3d_hull(all_points, simplices)

    # Dictionary to store the projected polygons (each simplex projected into the X-T plane) for each phase.
    phase_polygons = {}
    
    # Process each simplex from the convex hull.
    for simplex in simplices:
        # Get the 3D coordinates of the simplex vertices.
        simplex_points = all_points[simplex]
        # Project the simplex onto the X-T plane (i.e. drop the G component)
        projected_polygon = Polygon(simplex_points[:, :2])
        
        # Determine the phase label for this simplex by taking the majority label among its vertices.
        unique_phases = list(np.unique(phase_labels[simplex]))
        unique_phases.sort()
        phase = '_'.join(unique_phases)
        
        if projected_polygon.is_valid:
            phase_polygons.setdefault(phase, []).append(projected_polygon)
    
    # Merge overlapping polygons for each phase.
    for phase in phase_polygons:
        phase_polygons[phase] = unary_union(phase_polygons[phase])
    
    # Plot the final merged 2D phase regions with Matplotlib.
    fig, ax = plt.subplots(figsize=(8, 6))
    
    for phase, polygon in phase_polygons.items():
        color = color_dict.get(phase, 'gray')
        # polygon can be a Polygon or a MultiPolygon.
        if polygon.geom_type == 'Polygon':
            x, y = polygon.exterior.xy
            ax.fill(x, y, alpha=0.5, fc=color, ec='black', label=phase)
            
        if polygon.geom_type == 'MultiPolygon':
            for subpoly in polygon.geoms:
                x, y = subpoly.exterior.xy
                ax.fill(x, y, alpha=0.5, fc=color, ec='black', label=phase)
    
    ax.set_xlabel("X (FE)")
    ax.set_ylabel("Temperature (K)")
    ax.set_title("Phase Diagram (2D Projection of Convex Hull)")
    plt.show()

In [ ]:
# plot_phase_diagram(phase_dict, color_dict)

In [ ]:
fig = go.Figure()

for phase_name in phases:
    X, Y, Z = phase_dict[phase_name]


    X = X.reshape(10, -1)
    Y = Y.reshape(10, -1)
    Z = Z.reshape(10, -1)

    selected_color = color_dict[phase_name]
    # fig.add_trace(go.Scatter3d(
    #         x=X, y=Y, z=Z,
    #         mode='markers',
    #         name=phase_name,
    #         marker=dict(color=selected_color, size=1)
    #     ))
    
    fig.add_trace(go.Surface(x=X, y=Y, z=Z, 
                    colorscale=[[0, selected_color], [1, selected_color]],
                    showscale=False))


fig.update_layout(
    scene=dict(
        xaxis_title="X(FE)",
        yaxis_title="Temperature (K)",
        zaxis_title="Gibbs Energy (J/mol)"
    ),
    title="Gibbs Energy Surfaces"
)

fig.show()

### Then we are going to convert the functions to polynomials, calculate the boundary points, and plot the phase diagram